In [1]:
# Dependencies
from bs4 import BeautifulSoup
import requests
import pymongo
from splinter import Browser
import pandas as pd
from collections import OrderedDict

In [2]:
# Scrape for News
# URL of page to be scraped
url = 'https://mars.nasa.gov/news/'

# Retrieve page with the requests module
response = requests.get(url)
# Create BeautifulSoup object; parse with 'lxml'
soup = BeautifulSoup(response.text, 'lxml')

In [3]:
# Find title and body elements
results = soup.find('div', class_='slide')


# Identify and return title of article
article_title = results.find('div', class_='content_title').text
# Identify and return body preview of article
article_body = results.find('div', class_='rollover_description_inner').text

#Print statements
print(article_title)
print(article_body)




NASA Invites Students to Name Mars 2020 Rover



Through Nov. 1, K-12 students in the U.S. are encouraged to enter an essay contest to name NASA's next Mars rover.



In [4]:
# Scrape for Featured Image 
# https://splinter.readthedocs.io/en/latest/drivers/chrome.html
!which chromedriver

/usr/local/bin/chromedriver


In [5]:
executable_path = {'executable_path': '/usr/local/bin/chromedriver'}
browser = Browser('chrome', **executable_path, headless=False)

In [6]:
# Featured Image url
url = 'https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars'
browser.visit(url)

In [7]:
# HTML object
html = browser.html
# Parse HTML with Beautiful Soup
soup_html = BeautifulSoup(html, 'html.parser')

results = soup_html.find('div', class_='carousel_items')

In [8]:
# Print results to determine where img href is
print(results)

<div class="carousel_items">
<article alt="Animation of Ceres" class="carousel_item" style="background-image: url('/spaceimages/images/wallpaper/PIA19168-1920x1200.jpg');">
<div class="default floating_text_area ms-layer">
<h2 class="category_title">
</h2>
<h2 class="brand_title">
				  FEATURED IMAGE
				</h2>
<h1 class="media_feature_title">
				  Animation of Ceres				</h1>
<div class="description">
</div>
<footer>
<a class="button fancybox" data-description="NASA's Dawn spacecraft as it observed Ceres for an hour from a distance of 238,000 miles." data-fancybox-group="images" data-fancybox-href="/spaceimages/images/mediumsize/PIA19168_ip.jpg" data-link="/spaceimages/details.php?id=PIA19168" data-title="Animation of Ceres" id="full_image">
					FULL IMAGE
				  </a>
</footer>
</div>
<div class="gradient_container_top"></div>
<div class="gradient_container_bottom"></div>
</article>
</div>


In [9]:
# Parse footer for href
footer = results.footer
img = footer.find('a', class_='button fancybox')
img_href = img.get('data-fancybox-href')

In [10]:
# Add href to front url matter to give full url
featured_image_url = 'https://www.jpl.nasa.gov' + img_href
featured_image_url

'https://www.jpl.nasa.gov/spaceimages/images/mediumsize/PIA19168_ip.jpg'

In [11]:
# Scrape for Weather
# Visit url
url_weather = "https://twitter.com/marswxreport?lang=en"
browser.visit(url_weather)

In [12]:
# Grab newest tweet
html_weather = browser.html
soup_weather = BeautifulSoup(html_weather, "html.parser")
mars_weather = soup_weather.find("p", class_="TweetTextSize TweetTextSize--normal js-tweet-text tweet-text").text
print(mars_weather)

We won’t be hearing from @MarsCuriosity or @NASAInSight for the next 2 weeks during Mars solar conjunction. Read more about why Mars missions go silent every 2 years: https://www.wral.com/mars-spacecraft-go-quiet-during-solar-conjunction/18595551/ …pic.twitter.com/fWruE2v151


In [13]:
# Scrape for Facts
# Visit Facts url
url_facts = "https://space-facts.com/mars/"
browser.visit(url_facts)

In [14]:
# Read into table using Pandas
facts_table = pd.read_html(url_facts)
facts_table_df = facts_table[0]
facts_table_df

,Mars - Earth Comparison,Mars,Earth
0,Diameter:,"6,779 km","12,742 km"
1,Mass:,6.39 × 10^23 kg,5.97 × 10^24 kg
2,Moons:,2,1
3,Distance from Sun:,"227,943,824 km","149,598,262 km"
4,Length of Year:,687 Earth days,365.24 days
5,Temperature:,-153 to 20 °C,-88 to 58°C


In [15]:
# Convert table in html
table_html = facts_table_df.to_html(index=False)
print(table_html)

<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th>Mars - Earth Comparison</th>
      <th>Mars</th>
      <th>Earth</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <td>Diameter:</td>
      <td>6,779 km</td>
      <td>12,742 km</td>
    </tr>
    <tr>
      <td>Mass:</td>
      <td>6.39 × 10^23 kg</td>
      <td>5.97 × 10^24 kg</td>
    </tr>
    <tr>
      <td>Moons:</td>
      <td>2</td>
      <td>1</td>
    </tr>
    <tr>
      <td>Distance from Sun:</td>
      <td>227,943,824 km</td>
      <td>149,598,262 km</td>
    </tr>
    <tr>
      <td>Length of Year:</td>
      <td>687 Earth days</td>
      <td>365.24 days</td>
    </tr>
    <tr>
      <td>Temperature:</td>
      <td>-153 to 20 °C</td>
      <td>-88 to 58°C</td>
    </tr>
  </tbody>
</table>


In [16]:
# Scrape for Hemispheres (hem)
# Visit url
url_hem = "https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars"
browser.visit(url_hem)

In [17]:
h3 = browser.find_by_css('h3').first.value

In [18]:
html_hem = browser.html
soup_hem = BeautifulSoup(html_hem, "html.parser")
description = soup_hem.find("div", class_="collapsible results")


In [19]:
# Build List for all Links
hem_links = []
img_links = description.find_all('a', class_='itemLink product-item')

for link in img_links:
        # Use Beautiful Soup's find() method to navigate and retrieve attributes
        href = link['href']
        link_to_click = 'https://astrogeology.usgs.gov' + href
        hem_links.append(link_to_click)
        print(link_to_click)

https://astrogeology.usgs.gov/search/map/Mars/Viking/cerberus_enhanced
https://astrogeology.usgs.gov/search/map/Mars/Viking/cerberus_enhanced
https://astrogeology.usgs.gov/search/map/Mars/Viking/schiaparelli_enhanced
https://astrogeology.usgs.gov/search/map/Mars/Viking/schiaparelli_enhanced
https://astrogeology.usgs.gov/search/map/Mars/Viking/syrtis_major_enhanced
https://astrogeology.usgs.gov/search/map/Mars/Viking/syrtis_major_enhanced
https://astrogeology.usgs.gov/search/map/Mars/Viking/valles_marineris_enhanced
https://astrogeology.usgs.gov/search/map/Mars/Viking/valles_marineris_enhanced


In [20]:
# Drop Duplicate urls from List
hem_links = list(OrderedDict.fromkeys(hem_links))
print(hem_links)

['https://astrogeology.usgs.gov/search/map/Mars/Viking/cerberus_enhanced', 'https://astrogeology.usgs.gov/search/map/Mars/Viking/schiaparelli_enhanced', 'https://astrogeology.usgs.gov/search/map/Mars/Viking/syrtis_major_enhanced', 'https://astrogeology.usgs.gov/search/map/Mars/Viking/valles_marineris_enhanced']


In [26]:
# Create list to hold dictionaries of title and img url
hem_image_urls = []
# Loop through each Link
for href in hem_links:
    try: 
        browser.visit(href)
       
        html_hem_img = browser.html
    # Parse HTML with Beautiful Soup
        soup_hem_img = BeautifulSoup(html_hem_img, 'html.parser')
    # Retrieve all elements that contain image info
        img_info = soup_hem_img.find('section', class_='block metadata')
    
        title = img_info.find('h2', class_= 'title').text
        img_a_href = img_info.find('a')
        img_url = img_a_href.get('href')
        hem_image_urls.append({"title" : title, "img_url" : img_url})
        print('-----------')
        print(title)
        print(img_url)
        print(href)
        
    # Tab back to previous page for next link
        browser.back()
    except: print('e')

-----------
Cerberus Hemisphere Enhanced
http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif
https://astrogeology.usgs.gov/search/map/Mars/Viking/cerberus_enhanced
-----------
Schiaparelli Hemisphere Enhanced
http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif
https://astrogeology.usgs.gov/search/map/Mars/Viking/schiaparelli_enhanced
-----------
Syrtis Major Hemisphere Enhanced
http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/syrtis_major_enhanced.tif
https://astrogeology.usgs.gov/search/map/Mars/Viking/syrtis_major_enhanced
e


In [22]:
# Check list
print(hem_image_urls)

[]
